In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import h5py
from scipy import signal
import _pickle as pkl
import random
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras import layers, models, optimizers
import math
import imageio

In [4]:
ages = ['Age6-9', 'Age10-11', 'Age12-13', 'Age14-17', 'Age18-24', 'Age25-44']
dataset = {'Age6-9':{}, 'Age10-11':{}, 'Age12-13':{}, 'Age14-17':{}, 'Age18-24':{}, 'Age25-44':{}}

In [5]:
train_dir = r'D:\Jason\train_999'
test_dir = r'D:\Jason\test_999'

Save dataset as indiviual frames

In [5]:
n = np.arange(1, 1000)
delay = 0.25
frame = np.around(24 * (50 * n / 500))
for i in range(frame.shape[0]):
    if frame[i]<300:
        frame[i] = 0
    elif frame[i]>=300 and frame[i]<419:
        frame[i] = 1
    elif frame[i]>=419 and frame[i]<666:
        frame[i] = 0
    elif frame[i]>=666 and frame[i]<781:
        frame[i] = 1
    elif frame[i]>=781 and frame[i]<978:
        frame[i] = 0
    elif frame[i]>=978 and frame[i]<1085:
        frame[i] = 1
    elif frame[i]>=1085 and frame[i]<1509:
        frame[i] = 0
    elif frame[i]>=1509 and frame[i]<2096:
        frame[i] = 1
    elif frame[i]>=2096:
        frame[i] = 0

y = frame

In [7]:
DATA_DIR = 'D:\Jason'
dataset_dir = os.path.join(DATA_DIR, 'Dataset2')
dataset_file_name = os.path.join(dataset_dir, 'dataset.pkl')
with open(dataset_file_name, 'rb') as fp:
    dataset = pkl.load(fp)
os.makedirs(train_dir)
os.makedirs(test_dir)
np.random.seed(0)
ages = ['Age6-9', 'Age10-11', 'Age12-13', 'Age14-17', 'Age18-24', 'Age25-44']
data_size = 75
train_data, test_data = [], []
train_idxs = np.random.choice(data_size, int(0.8 * data_size), replace=False)
data_idx = 0
i = 0
for age in ages:
    i = 0
    for patient_key in dataset[age]:
        i = i+1
        if data_idx in train_idxs:
            train_data.append(dataset[age][patient_key])
        else:
            test_data.append(dataset[age][patient_key])
        try:
            assert(dataset[age][patient_key].shape==(999,111,129))
        except:
            print(age, i, dataset[age][patient_key].shape, patient_key)
        data_idx += 1
del dataset
labels = []
for i, patient_data in enumerate(train_data):
    print('train', i)
    patient_data[:, :, :10]=0
    patient_data = (patient_data - np.min(patient_data))/(np.max(patient_data)-np.min(patient_data))
    for j, image in enumerate(patient_data):
        image = (np.stack([image]*3, -1) * 255).astype(np.uint8)
        save_dir = '{}_{}.png'.format(str(i).zfill(2), str(j).zfill(4))
        imageio.imsave(os.path.join(train_dir, save_dir), image)
        labels.append(y[j])
open(os.path.join(train_dir, 'labels.txt'), 'w').write(','.join(np.array(labels).astype(str)))
labels = [] 
for i, patient_data in enumerate(test_data):
    patient_data[:, :, :10]=0
    patient_data = (patient_data - np.min(patient_data))/(np.max(patient_data)-np.min(patient_data))
    print('test', i)
    for j, image in enumerate(patient_data):
        image = (np.stack([image]*3, -1) * 255).astype(np.uint8)
        save_dir = '{}_{}.png'.format(str(i).zfill(2), str(j).zfill(4))
        imageio.imsave(os.path.join(test_dir, save_dir), image)
        labels.append(y[j])
open(os.path.join(test_dir, 'labels.txt'), 'w').write(','.join(np.array(labels).astype(str)))

Load dataset

In [8]:
dirs_train = np.array([os.path.join(train_dir, i) for i in os.listdir(train_dir) if i[-4:]=='.png'])
dirs_test = np.array([os.path.join(test_dir, i) for i in os.listdir(test_dir) if i[-4:]=='.png'])
y_train = np.array(open(os.path.join(train_dir, 'labels.txt')).read().split(',')).astype(float).astype(int)
y_oh = np.zeros((len(y_train), 2)).astype(int)
y_oh[np.arange(len(y_oh)), y_train] = 1
y_test = np.array(open(os.path.join(test_dir, 'labels.txt')).read().split(',')).astype(float).astype(int)
y_oh_test = np.zeros((len(y_test), 2)).astype(int)
y_oh_test[np.arange(len(y_oh_test)), y_test] = 1

In [9]:
def load_fn(path):
    x = tf.io.read_file(path)
    x = tf.io.decode_png(x, channels=3)
    x = tf.reshape(x, (111, 129, 3))
    x = tf.cast(x, tf.float32)
    return x

In [10]:
d_X = tf.data.Dataset.from_tensor_slices(dirs_train)
d_y = tf.data.Dataset.from_tensor_slices(y_oh)
d_X = d_X.map(load_fn)
train_data = tf.data.Dataset.zip((d_X, d_y))
train_data = train_data.shuffle(100)
train_data = train_data.repeat()
train_data = train_data.batch(64, drop_remainder=True)
d_X_test = tf.data.Dataset.from_tensor_slices(dirs_test)
d_X_test = d_X_test.map(load_fn)
d_y_test = tf.data.Dataset.from_tensor_slices(y_oh_test)
test_data = tf.data.Dataset.zip((d_X_test, d_y_test))

In [11]:
np.random.seed(0)
ages = ['Age6-9', 'Age10-11', 'Age12-13', 'Age14-17', 'Age18-24', 'Age25-44']
data_size = 75
train_data, test_data = [], []
train_idxs = np.random.choice(data_size, int(0.8 * data_size), replace=False)
data_idx = 0
i = 0
for age in ages:
    i = 0
    for patient_key in dataset[age]:
        i = i+1
        if data_idx in train_idxs:
            train_data.append(dataset[age][patient_key])
        else:
            test_data.append(dataset[age][patient_key])
        try:
            assert(dataset[age][patient_key].shape==(999,111,129))
        except:
            print(age, i, dataset[age][patient_key].shape, patient_key)
        data_idx += 1
del dataset

In [12]:
X_train = []
y_train = []
X_test = []
y_test = []
counter = 0
while train_data:
    x = train_data.pop(0)
    windows = x.shape[0]
    for i in range(windows):
        X_train.append((x[i] - x[i].min())/(x[i].max() - x[i].min()))
        y_train.append(y[i])
while test_data:
    x = test_data.pop(0)
    windows = x.shape[0]
    for i in range(windows):
        X_test.append((x[i] - x[i].min())/(x[i].max() - x[i].min()))
        y_test.append(y[i])

In [13]:
label_dict = {0: 'Human', 1: 'Logic'}
for each_x in X_train:
    try:
        assert(each_x.shape==(111,129))
    except:
        print(each_x.shape)
for each_x in X_test:
    try:
        assert(each_x.shape==(111,129))
    except:
        print(each_x.shape)
assert(len(X_train)==len(y_train))
assert(len(X_test)==len(y_test))
print(len(X_train), len(X_test))

In [14]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [15]:
per_train = np.random.permutation(len(X_train))
per_test = np.random.permutation(len(X_test))
X_train = X_train[per_train]
y_train = y_train[per_train]
X_test = X_test[per_test]
y_test = y_test[per_test]

In [16]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [17]:
# from tensorflow.keras.layers import Input
# vgg_model = tf.keras.applications.vgg16.VGG16(input_tensor = Input(shape=(111, 129, 3)))

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
with tf.device('/gpu:0'):
    print(tf.test.gpu_device_name())

/device:GPU:0


In [19]:
tf.test.is_built_with_cuda()

True

In [ ]:
input_layer = tf.keras.layers.Input(shape=(111, 129, 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(input_layer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs=input_layer, outputs=x)

In [58]:
base_model = tf.keras.applications.vgg19.VGG19(input_shape=(111, 129, 3), include_top=False, weights='imagenet')
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(111, 129, 3), include_top=False, weights='imagenet')
input_layer = tf.keras.layers.Input(shape=(111, 129, 3))
output_layer = tf.keras.layers.Dense(2, activation='softmax')
model = tf.keras.Sequential([input_layer, base_model, tf.keras.layers.GlobalAveragePooling2D(), output_layer])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 4, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 1280)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [59]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
model.fit(train_data, epochs=30, steps_per_epoch=1000, validation_data=test_data, validation_steps=100)